In [2]:
import os
import json
import pandas as pd
from tqdm import tqdm


from utils.prompts import generate_question_evaluation_prompt_v2
from utils.bedrock_functions import invoke_bedrock_endpoint, build_anthropic_request_body
from utils.bedrock_functions import build_mistral_request_body, build_nova_request_body, build_llama_request_body

from utils.misc_helpers import read_jsonl_file, save_as_jsonl


In [3]:
directory = "/home/ec2-user/preetam_experiments"
split = 'test'
data = json.load(open(f"{directory}/batch_processing_files/{split}/intermediate_res/qa_pairs_{split}.json"))

In [4]:
test = read_jsonl_file('/home/ec2-user/code_repos/dynamicKGQA/inputs/annotator_requests/tests/annotator_requests_qcheck_mistral_test.jsonl')

In [5]:
test[0]

{'recordId': '00009785500',
 'modelInput': {'modelId': 'mistral.mistral-small-2402-v1:0',
  'contentType': 'application/json',
  'accept': 'application/json',
  'body': '{"prompt": "<s>[INST] As an expert evaluator, your role is to assess the quality and validity of trivia or natural questions. These questions aim to test the responder\'s knowledge, which may require implicit or external information. Your goal is to analyze the question based on the following criteria:\\n\\n- **Logical Structure**: Verify if the grammar and syntax are correct. (True if grammatically and syntactically correct; False if there are issues with grammar or syntax.)\\n- **Redundancy**: Confirm that the question does not contain its own answer explicitly or through overly obvious phrasing. (True if it contains its answer; False if it does not.)\\n- **Multiple Answers**: Determine if the question allows for multiple valid answers. This is acceptable in some cases, but flag it if it reduces the question\'s effec

In [6]:
request_body = test[0]['modelInput']
response = invoke_bedrock_endpoint(
    request_body=request_body["body"],
    model_id=request_body["modelId"],
    contentType=request_body["contentType"]
)
print(response)

{'outputs': [{'text': ' {\n  "question": "What family does the alpaca belong to, considering its evolutionary lineage through its parent taxa?",\n  "logical_structure_flag": true,\n  "logical_structure_reasoning": "The question is grammatically and syntactically correct.",\n  "redundancy_flag": false,\n  "redundancy_reasoning": "The question does not contain its own answer explicitly or through overly obvious phrasing.",\n  "multiple_answers_flag": false,\n  "multiple_answers_reasoning": "The question is expected to have one valid answer, which is the taxonomic family to which the alpaca belongs."\n}', 'stop_reason': 'stop'}]}


In [7]:
test_var = []
for i in range(107):
    temp = test[i]
    temp['modelInput'] = {'recordId': temp['recordId'],
                          'modelInput': temp['modelInput']['body']}
    test_var.append(test[i]['modelInput'])

In [10]:
json.loads(test_var[0]['modelInput'])

{'prompt': '<s>[INST] As an expert evaluator, your role is to assess the quality and validity of trivia or natural questions. These questions aim to test the responder\'s knowledge, which may require implicit or external information. Your goal is to analyze the question based on the following criteria:\n\n- **Logical Structure**: Verify if the grammar and syntax are correct. (True if grammatically and syntactically correct; False if there are issues with grammar or syntax.)\n- **Redundancy**: Confirm that the question does not contain its own answer explicitly or through overly obvious phrasing. (True if it contains its answer; False if it does not.)\n- **Multiple Answers**: Determine if the question allows for multiple valid answers. This is acceptable in some cases, but flag it if it reduces the question\'s effectiveness or specificity. (True if multiple answers are plausible; False if only one valid answer is expected.)\n    \n#### Output JSON Keys:\n- `question`: The input question

In [11]:
save_as_jsonl(test_var, '/home/ec2-user/code_repos/dynamicKGQA/outputs/annotations/test/annotator_requests_qcheck_mistral_test100.jsonl')
